# Health Insurance Lead Prediction
> ### Background
The Client FinMan is a financial services company that provides various financial services like loan, investment funds, insurance etc. to its customers. FinMan wishes to cross-sell health insurance to the existing customers who may or may not hold insurance policies with the company. The company recommend health insurance to it's customers based on their profile once these customers land on the website. Customers might browse the recommended health insurance policy and consequently fill up a form to apply. When these customers fill-up the form, their Response towards the policy is considered positive and they are classified as a lead. Once these leads are acquired, the sales advisors approach them to convert and thus the company can sell proposed health insurance to these leads in a more efficient manner.

> ### Objective
Company requires a model to be built to predict whether the person will be interested in their proposed Health plan/policy given the information provided about the customer 

> ### Evaluation
The evaluation metric for model selection is `roc_auc_score` across all entries in the test set.

> ### Dataset
> #### Train Dataset
Training dataset is in `csv` format that contains 13 independent features, 1 dependent (target) feature and 50,882 number of rows
> #### Test Dataset
Test dataset is in `csv` format that contains 13 feature, and 21,805 number of rows. This dataset is to be used for model evaluations.
> #### Data Dictionary
|Variable|Definition|
|:---|:---|
|ID|Unique Identifier for a row|
|City_Code|Code for the City of the customers|
|Region_Code|Code for the Region of the customers|
|Accomodation_Type|Customer Owns or Rents the house|
|Reco_Insurance_Type|Joint or Individual type for the recommended insurance|  
|Upper_Age|Maximum age of the customer |
|lower _Age|Minimum age of the customer|
|is_Spouse|If the customers are married to each other (in case of joint insurance)|
|Health_Indicator|Encoded values for health of the customer|
|Holding_Policy_Duration|Duration (in years) of holding policy (a policy that customer has already subscribed to with the company)|
|Holding_Policy_Type|Type of holding policy|
|Reco_Policy_Cat|Encoded value for recommended health insurance
|Reco_Policy_Premium|Annual Premium (INR) for the recommended health insurance
|Response (Target)|0 : Customer did not show interest in the recommended policy <br>1 : Customer showed interest in the recommended policy|

## 1. Import required module

In [69]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## 2. Load the datasets

In [39]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

## 3. Data exploration and Data preprocessing

In [40]:
train_df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0


In [41]:
test_df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0
3,50886,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0
4,50887,C1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0


In [42]:
print(f'Training number of rows: {len(train_df)}')
print(f'Testing number of rows: {len(test_df)}')

Training number of rows: 50882
Testing number of rows: 21805


In [43]:
# Train set column info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       50882 non-null  int64  
 1   City_Code                50882 non-null  object 
 2   Region_Code              50882 non-null  int64  
 3   Accomodation_Type        50882 non-null  object 
 4   Reco_Insurance_Type      50882 non-null  object 
 5   Upper_Age                50882 non-null  int64  
 6   Lower_Age                50882 non-null  int64  
 7   Is_Spouse                50882 non-null  object 
 8   Health Indicator         39191 non-null  object 
 9   Holding_Policy_Duration  30631 non-null  object 
 10  Holding_Policy_Type      30631 non-null  float64
 11  Reco_Policy_Cat          50882 non-null  int64  
 12  Reco_Policy_Premium      50882 non-null  float64
 13  Response                 50882 non-null  int64  
dtypes: float64(2), int64(6

In [44]:
# Test set column info
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21805 entries, 0 to 21804
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       21805 non-null  int64  
 1   City_Code                21805 non-null  object 
 2   Region_Code              21805 non-null  int64  
 3   Accomodation_Type        21805 non-null  object 
 4   Reco_Insurance_Type      21805 non-null  object 
 5   Upper_Age                21805 non-null  int64  
 6   Lower_Age                21805 non-null  int64  
 7   Is_Spouse                21805 non-null  object 
 8   Health Indicator         16778 non-null  object 
 9   Holding_Policy_Duration  13202 non-null  object 
 10  Holding_Policy_Type      13202 non-null  float64
 11  Reco_Policy_Cat          21805 non-null  int64  
 12  Reco_Policy_Premium      21805 non-null  float64
dtypes: float64(2), int64(5), object(6)
memory usage: 2.2+ MB


In [45]:
# Health Indicator is missing an underscore in it. Lets fix it to make is consistent with rest of the columns
train_df.rename(columns={'Health Indicator':'Health_Indicator'}, inplace=True)
test_df.rename(columns={'Health Indicator':'Health_Indicator'}, inplace=True)

In [46]:
# Holding_Policy_Duration is actually in years, change data-type from object to float64
# First check if we have any unusual values that can't be converted to float
train_df['Holding_Policy_Duration'].unique() 

array(['14+', nan, '1.0', '3.0', '5.0', '9.0', '14.0', '7.0', '2.0',
       '11.0', '10.0', '8.0', '6.0', '4.0', '13.0', '12.0'], dtype=object)

In [47]:
# Need to convert 14+ to 14
train_df['Holding_Policy_Duration'].replace('14+', '14', inplace=True)
test_df['Holding_Policy_Duration'].replace('14+', '14', inplace=True) 
train_df['Holding_Policy_Duration'].unique() 

array(['14', nan, '1.0', '3.0', '5.0', '9.0', '14.0', '7.0', '2.0',
       '11.0', '10.0', '8.0', '6.0', '4.0', '13.0', '12.0'], dtype=object)

In [48]:
# Change data-type from object to float64
print(train_df['Holding_Policy_Duration'].dtype)
train_df['Holding_Policy_Duration'] = train_df['Holding_Policy_Duration'].astype('float64')
test_df['Holding_Policy_Duration'] = test_df['Holding_Policy_Duration'].astype('float64')
print(train_df['Holding_Policy_Duration'].dtype)

object
float64


In [49]:
# Train set stats
train_df.describe()

,ID,Region_Code,Upper_Age,Lower_Age,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
count,50882.000000,50882.000000,50882.000000,50882.000000,30631.000000,30631.000000,50882.000000,50882.000000,50882.000000
mean,25441.500000,1732.788707,44.856275,42.738866,6.015246,2.439228,15.115188,14183.950069,0.239947
std,14688.512535,1424.081652,17.310271,17.319375,4.501648,1.025923,6.340663,6590.074873,0.427055
min,1.000000,1.000000,18.000000,16.000000,1.000000,1.000000,1.000000,2280.000000,0.000000
25%,12721.250000,523.000000,28.000000,27.000000,2.000000,1.000000,12.000000,9248.000000,0.000000
50%,25441.500000,1391.000000,44.000000,40.000000,5.000000,3.000000,17.000000,13178.000000,0.000000
75%,38161.750000,2667.000000,59.000000,57.000000,9.000000,3.000000,20.000000,18096.000000,0.000000
max,50882.000000,6194.000000,75.000000,75.000000,14.000000,4.000000,22.000000,43350.400000,1.000000


In [50]:
# Test set stats
train_df.describe()

,ID,Region_Code,Upper_Age,Lower_Age,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
count,50882.000000,50882.000000,50882.000000,50882.000000,30631.000000,30631.000000,50882.000000,50882.000000,50882.000000
mean,25441.500000,1732.788707,44.856275,42.738866,6.015246,2.439228,15.115188,14183.950069,0.239947
std,14688.512535,1424.081652,17.310271,17.319375,4.501648,1.025923,6.340663,6590.074873,0.427055
min,1.000000,1.000000,18.000000,16.000000,1.000000,1.000000,1.000000,2280.000000,0.000000
25%,12721.250000,523.000000,28.000000,27.000000,2.000000,1.000000,12.000000,9248.000000,0.000000
50%,25441.500000,1391.000000,44.000000,40.000000,5.000000,3.000000,17.000000,13178.000000,0.000000
75%,38161.750000,2667.000000,59.000000,57.000000,9.000000,3.000000,20.000000,18096.000000,0.000000
max,50882.000000,6194.000000,75.000000,75.000000,14.000000,4.000000,22.000000,43350.400000,1.000000


### Handle Missing Values

In [51]:
# Check for missing values
print('Train Set')
print('=========')
print(train_df.isnull().sum())
print()
print('Test Set')
print('========')
print(test_df.isnull().sum())

Train Set
ID                             0
City_Code                      0
Region_Code                    0
Accomodation_Type              0
Reco_Insurance_Type            0
Upper_Age                      0
Lower_Age                      0
Is_Spouse                      0
Health_Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
Reco_Policy_Cat                0
Reco_Policy_Premium            0
Response                       0
dtype: int64

Test Set
ID                            0
City_Code                     0
Region_Code                   0
Accomodation_Type             0
Reco_Insurance_Type           0
Upper_Age                     0
Lower_Age                     0
Is_Spouse                     0
Health_Indicator           5027
Holding_Policy_Duration    8603
Holding_Policy_Type        8603
Reco_Policy_Cat               0
Reco_Policy_Premium           0
dtype: int64


In [52]:
# Missing values percentage
train_missing = round(train_df.isnull().sum()/len(train_df) * 100, 2) 
test_missing = round(test_df.isnull().sum()/len(test_df) * 100, 2) 
print('Train Set')
print('=========')
print(train_missing[train_missing > 0])
print()
print('Test Set')
print('========')
print(test_missing[test_missing > 0])

Train Set
Health_Indicator           22.98
Holding_Policy_Duration    39.80
Holding_Policy_Type        39.80
dtype: float64

Test Set
Health_Indicator           23.05
Holding_Policy_Duration    39.45
Holding_Policy_Type        39.45
dtype: float64


`Health_Indicator`, `Holding_Policy_type`, `Holding_Policy_Duration` has aproximately 23%, 39% and 39% values missing in both train and test set

In [53]:
# Health_Indicator
print(train_df['Health_Indicator'].unique())
print(test_df['Health_Indicator'].unique())

['X1' 'X2' nan 'X4' 'X3' 'X6' 'X5' 'X8' 'X7' 'X9']
[nan 'X1' 'X3' 'X2' 'X5' 'X4' 'X7' 'X6' 'X9' 'X8']


there are 9 unique values for health indicator

In [54]:
# Holding_Policy_Type
print(train_df['Holding_Policy_Type'].unique())
print(test_df['Holding_Policy_Type'].unique())

[ 3. nan  1.  4.  2.]
[ 3.  4. nan  1.  2.]


there are 4 unique values for holding

In [55]:
# Impute categorical features "Health_Indicator" & "Holding_Policy_Type" by their respective 'mode'`
train_df['Health_Indicator'].fillna(train_df['Health_Indicator'].mode()[0], inplace=True)
test_df['Health_Indicator'].fillna(test_df['Health_Indicator'].mode()[0], inplace=True)
train_df['Holding_Policy_Type'].fillna(train_df['Holding_Policy_Type'].mode()[0], inplace=True)
test_df['Holding_Policy_Type'].fillna(test_df['Holding_Policy_Type'].mode()[0], inplace=True)

In [56]:
# Impute numerical feature "Holding_Policy_Duration" by its 'mean'
train_df['Holding_Policy_Duration'].fillna(train_df['Holding_Policy_Duration'].mean(), inplace=True)
test_df['Holding_Policy_Duration'].fillna(test_df['Holding_Policy_Duration'].mean(), inplace=True)

In [57]:
# Check for missing values again 
print('Train Set')
print('=========')
print(train_df.isnull().sum())
print()
print('Test Set')
print('========')
print(test_df.isnull().sum())

Train Set
ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Upper_Age                  0
Lower_Age                  0
Is_Spouse                  0
Health_Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Reco_Policy_Premium        0
Response                   0
dtype: int64

Test Set
ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Upper_Age                  0
Lower_Age                  0
Is_Spouse                  0
Health_Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Reco_Policy_Premium        0
dtype: int64


### Encode Categorical Features and Scale data

In [58]:
# I dentify categorical columns
cat_cols = train_df.select_dtypes(include=['object']).columns

 `Region_Code`, `Reco_Policy_Cat` and `Holding_Policy_Type` also appears to be categorical features but they contain numerical values. Lets examine further

In [59]:
# Convert Region_Code and Reco_Policy_Cat data-type to object
cols = ['Region_Code', 'Reco_Policy_Cat', 'Holding_Policy_Type']
train_df[cols] = train_df[cols].astype('object')
test_df[cols] = test_df[cols].astype('object')

In [60]:
# Unique Region_code values
train_df['Region_Code'].nunique()

5316

`Region_Code` is categorical but has around 5316 unique values

In [61]:
# Unique Reco_Policy_Cat values
train_df['Reco_Policy_Cat'].nunique()

22

`Reco_Policy_Cat` is categorical and has around 22 unique values

In [62]:
# Unique Reco_Policy_Cat values
train_df['Holding_Policy_Type'].nunique()

4

In [63]:
# Create a list of categorical columns
cat_cols = list(cat_cols)
cat_cols = cat_cols + ['Reco_Policy_Cat', 'Holding_Policy_Type', 'Region_Code']
cat_cols

['City_Code',
 'Accomodation_Type',
 'Reco_Insurance_Type',
 'Is_Spouse',
 'Health_Indicator',
 'Reco_Policy_Cat',
 'Holding_Policy_Type',
 'Region_Code']

In [64]:
# Prepare the list of columns to be one-hot-encoded, Region code has over 5000 values hence it'd label encoded
one_hot_cols = cat_cols[:-1]
one_hot_cols

['City_Code',
 'Accomodation_Type',
 'Reco_Insurance_Type',
 'Is_Spouse',
 'Health_Indicator',
 'Reco_Policy_Cat',
 'Holding_Policy_Type']

In [65]:
# Create a list of numeric columns
numeric_cols = train_df.select_dtypes(include=['number']).columns
numeric_cols

Index(['ID', 'Upper_Age', 'Lower_Age', 'Holding_Policy_Duration',
       'Reco_Policy_Premium', 'Response'],
      dtype='object')

In [66]:
# We don't want ID and Response
numeric_cols = numeric_cols[1:-1]
numeric_cols

Index(['Upper_Age', 'Lower_Age', 'Holding_Policy_Duration',
       'Reco_Policy_Premium'],
      dtype='object')

In [31]:
# TRANSFORM DATA
  # 1. Label encode encode Region_Code column
  # 2. One hot encode rest of categorical columns 
  # 3. Normalize numeric columns 

scaler = ("minmax_scaler", MinMaxScaler(), numeric_cols)
one_hot_encoder = ("onehot_enc", OneHotEncoder(), one_hot_cols)
label_encoder = ("label_enc", OrdinalEncoder(), ['Region_Code'])
transformer = ColumnTransformer([label_encoder, scaler, one_hot_encoder], remainder='passthrough')

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,0.315789,0.338983,1.000000,0.227609,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2761.0,1.0,0.0
1,1.000000,0.101695,0.385788,0.687356,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,932.0,2.0,0.0
2,0.245614,0.271186,0.000000,0.125881,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3241.0,3.0,1.0
3,0.596491,0.542373,1.000000,0.377401,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3842.0,4.0,0.0
4,0.456140,0.474576,0.153846,0.197807,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1866.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50877,0.070175,0.101695,0.385788,0.132066,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,685.0,50878.0,0.0
50878,0.157895,0.186441,0.461538,0.076162,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3666.0,50879.0,0.0
50879,0.789474,0.796610,1.000000,0.221425,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,358.0,50880.0,0.0
50880,0.929825,0.559322,0.076923,0.630605,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,50881.0,1.0


## 4. Model building